### Import Libraries

In [1]:
import os
import json
import pandas as pd
import torch
from pathlib import Path
from typing import Dict, Any, List, Optional, Tuple, Set

from sentence_transformers import SentenceTransformer
from product_search.evaluator import SearchEvaluator
from product_search.benchmark import retrieve_bm25, retrieve_hnsw, retrieve_hyrid_rrf
from product_search.inference import OpenSearchInference

project_dir = Path(os.getcwd()).parent
data_dir = project_dir / "Data" / "RAW"
processed_dir = project_dir / "Data" / "PROCESSED"

print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

/home/harini/miniconda3/envs/product_search_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
GPU Name: NVIDIA GeForce RTX 5070


In [2]:
def load_embedder(model_name: str) -> tuple[SentenceTransformer, str]:
    """
    Returns (embedder, device_str).
    Prefers Apple Silicon GPU via MPS when available.
    """
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        device = "mps"
    elif torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"

    embedder = SentenceTransformer(model_name, device=device)
    return embedder, device

In [3]:
batch_size = 1000
ks_list = [5, 10, 20]
top_k = 20
binary_threshold = None
encode_batch_size=1024

fine_tuned_model_path = str(project_dir / 'src' / 'product_search' / 'finetuned_encoder' / 'finetuned_model')
base_embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
vector_field = "embedding"

evaluator = SearchEvaluator(ks=ks_list)

In [4]:
embedder, device = load_embedder(fine_tuned_model_path)
print(f"SentenceTransformer device = {device}")

SentenceTransformer device = cuda


In [5]:
inference_client = OpenSearchInference(
    bm25_index='products_bm25',
    hnsw_index='products_hnsw_finetuned',
    embedding_model=embedder
)

In [6]:
# load datasets
with open(processed_dir / "test_qrels.json", "r", encoding="utf-8") as f:
    gt_qrels_raw = json.load(f)

test_queries = pd.read_parquet(processed_dir / "test_query_table.parquet")

### BM25 Benchmarking

In [8]:
bm25_ranked = retrieve_bm25(
    inference_client,
    index="products_bm25",
    query_table=test_queries,
    topn=top_k,
    filter_source=None,
)

BM25 retrieval: 100%|██████████████████████████████████████████████████| 9100/9100 [00:26<00:00, 347.78it/s]


In [9]:
bm25_scores = evaluator.evaluate_rankings(
    ground_truth_qrels=gt_qrels_raw,
    predicted_rankings=bm25_ranked,
    binary_threshold=binary_threshold,
)

In [10]:
bm25_scores

,metric,k,score
0,mrr,5,0.564734
1,mrr,10,0.573447
2,mrr,20,0.577376
3,ndcg,5,0.370273
4,ndcg,10,0.334484
5,ndcg,20,0.327337
6,recall,5,0.131303
7,recall,10,0.200666
8,recall,20,0.278610


### HNSW Benchmarking

#### HNSW Benchmarking with Base Encoder

In [11]:
embedder, device = load_embedder(base_embedding_model_name)
print(f"SentenceTransformer device = {device}")

SentenceTransformer device = cuda


In [12]:
hnsw_ranked = retrieve_hnsw(
    inference_client,
    index="products_hnsw",
    query_table=test_queries,
    embedder=embedder,
    topn=top_k,
    filter_source=None,
    encode_batch_size=encode_batch_size,
)

HNSW retrieval: 100%|██████████████████████████████████████████████████| 9100/9100 [00:46<00:00, 195.92it/s]


In [13]:
hnsw_scores = evaluator.evaluate_rankings(
    ground_truth_qrels=gt_qrels_raw,
    predicted_rankings=hnsw_ranked,
    binary_threshold=binary_threshold,
)

In [14]:
hnsw_scores

,metric,k,score
0,mrr,5,0.506956
1,mrr,10,0.517510
2,mrr,20,0.521683
3,ndcg,5,0.323082
4,ndcg,10,0.294775
5,ndcg,20,0.291780
6,recall,5,0.111969
7,recall,10,0.174834
8,recall,20,0.249690


#### HNSW Benchmarking with Fine-tuned Encoder

In [21]:
embedder, device = load_embedder(fine_tuned_model_path)
print(f"SentenceTransformer device = {device}")

SentenceTransformer device = cuda


In [22]:
hnsw_ranked = retrieve_hnsw(
    inference_client,
    index="products_hnsw_finetuned",
    query_table=test_queries,
    embedder=embedder,
    topn=top_k,
    filter_source=None,
    encode_batch_size=encode_batch_size,
)

HNSW retrieval: 100%|██████████████████████████████████████████████████| 9100/9100 [00:46<00:00, 196.15it/s]


In [23]:
hnsw_scores = evaluator.evaluate_rankings(
    ground_truth_qrels=gt_qrels_raw,
    predicted_rankings=hnsw_ranked,
    binary_threshold=binary_threshold,
)

In [24]:
hnsw_scores

,metric,k,score
0,mrr,5,0.560949
1,mrr,10,0.571161
2,mrr,20,0.575117
3,ndcg,5,0.372501
4,ndcg,10,0.341996
5,ndcg,20,0.338934
6,recall,5,0.128919
7,recall,10,0.203652
8,recall,20,0.291662


### Hybrid (BM25 + HNSW) with RRF Benchmarking

In [7]:
embedder, device = load_embedder(fine_tuned_model_path)
print(f"SentenceTransformer device = {device}")

SentenceTransformer device = cuda


In [8]:
hybrid_ranked = retrieve_hyrid_rrf(
    inference_client,
    index_bm25="products_bm25",
    index_hnsw='products_hnsw_finetuned',
    query_table=test_queries,
    embedder=embedder,
    topn=top_k,
    filter_source=None,
    candidate_pool_size=30,
    encode_batch_size=encode_batch_size
)

Hybrid RRF retrieval: 100%|████████████████████████████████████████████| 9100/9100 [01:24<00:00, 107.89it/s]


In [9]:
hybrid_rrf_scores = evaluator.evaluate_rankings(
    ground_truth_qrels=gt_qrels_raw,
    predicted_rankings=hybrid_ranked,
    binary_threshold=binary_threshold,
)

In [10]:
hybrid_rrf_scores

,metric,k,score
0,mrr,5,0.602973
1,mrr,10,0.612671
2,mrr,20,0.616714
3,ndcg,5,0.400977
4,ndcg,10,0.368689
5,ndcg,20,0.365942
6,recall,5,0.141201
7,recall,10,0.223343
8,recall,20,0.318355


### Single Query Inference

In [7]:
embedder, device = load_embedder(fine_tuned_model_path)
print(f"SentenceTransformer device = {device}")

SentenceTransformer device = cuda


In [8]:
resp = inference_client.query_bm25(
    query='red puma socks',
    index='products_bm25',
    k=10, 
    include_full_text=True
)
resp

[SearchHit(product_id='amz_B07KX4BLQJ', score=30.021297, full_text='PUMA mens Team Liga Socks, Puma Blackpuma White, 3.5-6 US PUMA', source='ESCI', metadata={'product_title': 'PUMA mens Team Liga Socks, Puma Blackpuma White, 3.5-6 US', 'product_description': None, 'product_bullet_point': 'Knitted mesh for breathability\nPercent cotton heel and toe for comfort\nRight &left anatomically knitted foot-bed for left & right and size\n87% Polyester 12% Polypropylene 1% Elastane', 'product_brand': 'PUMA'}),
 SearchHit(product_id='amz_B08189MHRM', score=28.865574, full_text="PUMA Women's P112201 Quarter 6-Pack Socks PUMA", source='ESCI', metadata={'product_title': "PUMA Women's P112201 Quarter 6-Pack Socks", 'product_description': None, 'product_bullet_point': None, 'product_brand': 'PUMA'}),
 SearchHit(product_id='amz_B08189N4P5', score=28.865574, full_text="PUMA Women's P113578 Quarter 6 Pack Socks PUMA", source='ESCI', metadata={'product_title': "PUMA Women's P113578 Quarter 6 Pack Socks", '

In [9]:
resp = inference_client.query_hnsw(
    query='red puma mens socks',
    index='products_hnsw',
    k=10, 
    include_full_text=True
)
resp

[SearchHit(product_id='amz_B07NNTCV5K', score=0.82375765, full_text="PUMA Men's 8 Pack Low Cut Running Socks, Black, 10 1 US PUMA", source='ESCI', metadata={'product_title': "PUMA Men's 8 Pack Low Cut Running Socks, Black, 10 1 US", 'product_description': None, 'product_bullet_point': None, 'product_brand': 'PUMA'}),
 SearchHit(product_id='amz_B081XW2PKV', score=0.8146829, full_text="PUMA Men's CAT Tee, high Risk red, L PUMA", source='ESCI', metadata={'product_title': "PUMA Men's CAT Tee, high Risk red, L", 'product_description': None, 'product_bullet_point': 'Regular fit\nShort sleeve construction\nRib crew neck\nPUMA brand logo\nCotton and elastane materials', 'product_brand': 'PUMA'}),
 SearchHit(product_id='amz_B07PXTWJFY', score=0.8123556, full_text="PUMA Men's 8 Pack Low Cut Socks, White/Grey, 10-13 PUMA", source='ESCI', metadata={'product_title': "PUMA Men's 8 Pack Low Cut Socks, White/Grey, 10-13", 'product_description': None, 'product_bullet_point': 'Comfort toe\nArch support\

In [10]:
resp = inference_client.query_hnsw(
    query='red puma mens socks',
    index='products_hnsw_finetuned',
    k=10, 
    include_full_text=True)
resp

[SearchHit(product_id='amz_B07KX4BLQJ', score=0.9168885, full_text='PUMA mens Team Liga Socks, Puma Blackpuma White, 3.5-6 US PUMA', source='ESCI', metadata={'product_title': 'PUMA mens Team Liga Socks, Puma Blackpuma White, 3.5-6 US', 'product_description': None, 'product_bullet_point': 'Knitted mesh for breathability\nPercent cotton heel and toe for comfort\nRight &left anatomically knitted foot-bed for left & right and size\n87% Polyester 12% Polypropylene 1% Elastane', 'product_brand': 'PUMA'}),
 SearchHit(product_id='amz_B07NNTCV5K', score=0.9135776, full_text="PUMA Men's 8 Pack Low Cut Running Socks, Black, 10 1 US PUMA", source='ESCI', metadata={'product_title': "PUMA Men's 8 Pack Low Cut Running Socks, Black, 10 1 US", 'product_description': None, 'product_bullet_point': None, 'product_brand': 'PUMA'}),
 SearchHit(product_id='amz_B07PXTWJFY', score=0.9124218, full_text="PUMA Men's 8 Pack Low Cut Socks, White/Grey, 10-13 PUMA", source='ESCI', metadata={'product_title': "PUMA Men

In [12]:
resp = inference_client.query_hybrid_rrf(
    query='red puma mens socks',
    k=10,
    candidate_pool_size=30,
    include_full_text=True
)
resp

[SearchHit(product_id='amz_B07KX4BLQJ', score=0.03278688524590164, full_text='PUMA mens Team Liga Socks, Puma Blackpuma White, 3.5-6 US PUMA', source='ESCI', metadata={'product_title': 'PUMA mens Team Liga Socks, Puma Blackpuma White, 3.5-6 US', 'product_description': None, 'product_bullet_point': 'Knitted mesh for breathability\nPercent cotton heel and toe for comfort\nRight &left anatomically knitted foot-bed for left & right and size\n87% Polyester 12% Polypropylene 1% Elastane', 'product_brand': 'PUMA'}),
 SearchHit(product_id='amz_B07D1BQ9F2', score=0.03125, full_text='Puma 6-Pack No Show Mens Socks Stay-Up Cuff and Heel Cushioned Arch Support, White/Black/Red, 10-13 PUMA', source='ESCI', metadata={'product_title': 'Puma 6-Pack No Show Mens Socks Stay-Up Cuff and Heel Cushioned Arch Support, White/Black/Red, 10-13', 'product_description': None, 'product_bullet_point': None, 'product_brand': 'PUMA'}),
 SearchHit(product_id='amz_B012AP0ZR0', score=0.03076923076923077, full_text='PUM

In [12]:
[[1], [2]]

[[1], [2]]